In [ ]:
#发布者
import multiprocessing
import threading
import time
from paho.mqtt import client as mqtt_client

host = '127.0.0.1'
port = 1883

class MqttClient(threading.Thread):
    def __init__(self, threadid, host, port):
        threading.Thread.__init__(self)
        self.threadid = threadid
        self.host = host
        self.port = port

        def on_connect(client, userdata, flags, rc):
            if rc == 0:
                pass
            else:
                print(threadid, "Failed, code %d\n", rc)
        self.ckientGrop = []
        #=============[ ADD ]##################
        for i in range(0, 50):
            client = mqtt_client.Client()
            client.on_connect = on_connect
            client.connect(host, port)
            client.loop_start()
            self.ckientGrop.append(client)

    def run(self):
        while True:
            time.sleep(1)
            for i, client in enumerate(self.ckientGrop):
                client.publish("/public/" + str(self.threadid) + str(i), str(self.threadid))

if __name__ == '__main__':
    print("==================Test=================")
    publist = []
    for i in range(0, 20):
        publist.append(MqttClient(i, host, port))
        
    for ch in publist:
        ch.start()

In [ ]:
#订阅者
import threading
import time
from paho.mqtt import client as mqtt_client

host = '127.0.0.1'
#host = '43.136.133.43'
port = 1883


class MqttClient(threading.Thread):
    def __init__(self, threadid, host, port):
        threading.Thread.__init__(self)
        self.threadid = threadid
        self.host = host
        self.port = port

        def on_connect(client, userdata, flags, rc):
            if rc == 0:
                pass
            else:
                print(threadid, "Failed, code %d\n", rc)
        self.ckientGrop = []
        #=============[ ADD ]##################
        for i in range(0, 10):
            client = mqtt_client.Client()
            client.on_connect = on_connect
            client.connect(host, port)
            client.loop_start()
            self.ckientGrop.append(client)

    def disconnect(self):
        for client in self.ckientGrop:
            client.disconnect()

    def run(self):
        def on_message(client, userdata, msg):
            pass
        for i, client in enumerate(self.ckientGrop):
            client.subscribe("/public/" + str(self.threadid) + str(i))
            client.on_message = on_message


if __name__ == '__main__':
    print("==================Connect=================")
    sublist = []
    for i in range(0, 400):
        sublist.append(MqttClient(i, host, port))
    print("==================START=================")
    for ch in sublist:
        ch.start()
